In [ ]:
pip install langgraph openpyxl pandas pydantic

In [ ]:
pip install -U langgraph "langchain[anthropic]"

In [16]:
import os

ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

In [ ]:
import pandas as pd
from pydantic import BaseModel

class ExcelAnalysis(BaseModel):
    total_sum: float
    average: float

def analyze_excel(file_path: str) -> ExcelAnalysis:
    """
    Reads an Excel file from the given file path, calculates the sum and average
    of all numeric values in the file, and returns them as a structured response.
    """
    df = pd.read_excel(file_path, header=None)
    df_numeric = df.apply(pd.to_numeric, errors='coerce')
    all_numbers = df_numeric.values.flatten()
    all_numbers = [x for x in all_numbers if pd.notna(x)]

    total_sum = sum(all_numbers)
    average = sum(all_numbers) / len(all_numbers) if len(all_numbers) > 0 else 0
    return ExcelAnalysis(total_sum=total_sum, average=average)

In [ ]:
def get_text_from_response(response):

    for msg in reversed(response['messages']):

        if msg.__class__.__name__ == "AIMessage":
            content = getattr(msg, "content", None)

            if isinstance(content, str):
                return content
            elif isinstance(content, list):

                texts = [item.get("text") for item in content if "text" in item]
                return " ".join(texts)
    return "No response found."

In [ ]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

agent = create_react_agent(
    model="anthropic:claude-3-7-sonnet-latest",
    tools=[analyze_excel],
    checkpointer=checkpointer
)

In [15]:
from google.colab import files
import pandas as pd

config = {"configurable": {"user_name": "Maor", "thread_id": "1"}}

print("Agent: Hi Maor! I am your Excel assistant. You can ask questions or upload Excel files.")

def safe_analyze_excel(file_path: str):
    """Analyze Excel file and handle empty files gracefully."""
    df = pd.read_excel(file_path)
    if df.empty:
        return None
    total_sum = df.sum(numeric_only=True).sum()
    count = df.select_dtypes(include='number').count().sum()
    average = total_sum / count if count > 0 else 0
    return total_sum, average

while True:
    user_input = input("You: ").strip()

    if user_input.lower() in ["exit", "quit", "thank you", "thanks", "that's all", "thank you!"]:
        print("Agent: You're welcome! Goodbye!")
        break

    response = agent.invoke(
        {"messages": [{"role": "user", "content": user_input}]},
        config
    )

    agent_reply = get_text_from_response(response)
    print("Agent:", agent_reply)

    if "file path" in agent_reply.lower() or "upload" in agent_reply.lower():
        while True:
            print("Agent: Please select the Excel file you want me to analyze...")
            uploaded = files.upload()

            if len(uploaded) == 0:
                print("Agent: No file uploaded. What would you like to do?")
                choice = input("Type 'retry' to upload again, or 'finish' to end: ").strip().lower()
                if choice == "Finish":
                    print("Agent: Okay, ending the session. Goodbye!")
                    exit()
                elif choice == "Retry":
                    continue
                else:
                    print("Agent: I didn't understand. Let's try uploading again.")
                    continue

            file_name = list(uploaded.keys())[0]

            if not file_name.lower().endswith((".xlsx", ".xls")):
                print(f"Agent: Sorry, '{file_name}' is not an Excel file. Please upload a .xlsx or .xls file.")
                continue

            analysis = safe_analyze_excel(file_name)
            if analysis is None:
                print(f"Agent: The file '{file_name}' is empty. Please upload a non-empty Excel file.")
                continue

            total_sum, average = analysis
            print(f"Agent: Got the file {file_name}, analyzing now...")
            print(f"Agent: The sum is {total_sum} and the average is {average:.2f}")
            break

Agent: Hi Maor! I am your Excel assistant. You can ask questions or upload Excel files.
Agent: I'd be happy to help you calculate the sum and average of your Excel file. However, I still need you to provide the file path of the Excel file you want to analyze.

When you upload a file, please share the complete file path where the Excel file is located. Without this information, I cannot access the file to perform the analysis.

For example, if your file is named "data.xlsx" and is located in your Documents folder, the path might look like:
- On Windows: "C:\Users\YourName\Documents\data.xlsx"
- On Mac: "/Users/YourName/Documents/data.xlsx"

Please provide the file path, and I'll immediately analyze your Excel file to calculate the sum and average of all numeric values.
Agent: Please select the Excel file you want me to analyze...


Saving Numbers.xlsx to Numbers (11).xlsx
Agent: Got the file Numbers (11).xlsx, analyzing now...
Agent: The sum is 4525.0 and the average is 167.59


KeyboardInterrupt: Interrupted by user